In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers --quiet
!pip install sentence_transformers --quiet
!pip install tqdm --quiet
!pip install datasets --quiet
!pip install evaluate --quiet
!pip install peft --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.4 MB/s eta 0:0

In [ ]:
!pip install emoji --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import re
import emoji
import argparse
import os
import evaluate
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader,Dataset
import os

import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from datasets import Dataset
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup

from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType

In [ ]:
lr = 1e-3
num_epochs = 3
batch_size = 8
device = "cuda"
model_name_or_path = "google/flan-t5-large"
tokenizer_name_or_path = "google/flan-t5-large"


In [ ]:


# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 2359296 || all params: 785509376 || trainable%: 0.30035236651331837


In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
import os
path = r"/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/*.csv" # use your path
all_files = glob.glob(path)
all_files

['/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2011Tornado_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2016Matthew_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2018Wildfires_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2013Floods_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2012Sandy_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2017Hurricane_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2016Blizzard_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2018Michael_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2019Dorian_Summary.csv']

In [ ]:
filenames=[]
for p in all_files:
    filenames.append(p.split("/")[-1])
    
df_dict = dict()
for n,f in zip(filenames,all_files):
    df_dict[n] = pd.read_csv(f)
    if n=='2018Michael_Summary.csv':
      df_dict[n]['num']=df_dict[n]['Unnamed: 0']
      df_dict[n]['label']=df_dict[n]['sentiment']
      df_dict[n].drop(['sentiment','Unnamed: 0'],axis=1,inplace=True)
    df_dict[n]['filename']=[n]*len(df_dict[n])
   

In [ ]:
df_dict.keys()

dict_keys(['2011Tornado_Summary.csv', '2016Matthew_Summary.csv', '2018Wildfires_Summary.csv', '2013Floods_Summary.csv', '2012Sandy_Summary.csv', '2017Hurricane_Summary.csv', '2016Blizzard_Summary.csv', '2018Michael_Summary.csv', '2019Dorian_Summary.csv'])

In [ ]:
df_all=pd.concat(df_dict.values(), ignore_index=True) 
required_df=df_all[['text','label','filename']]
required_df.head()
id2label={0:"Negative",1:"Positive"}
required_df=required_df.replace({"label": id2label})
def clean_text(line):
  clean_text=re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*','',line)
  clean_text = clean_text.lower()
  clean_text = re.sub("[^a-z0-9]"," ", clean_text)
  clean_text = re.sub("@[A-Za-z0-9]+","",clean_text) #Remove @ sign
  clean_text = " ".join(clean_text.split())
  clean_text = ''.join(c for c in clean_text if c not in emoji.EMOJI_DATA) #Remove Emojis
  return clean_text
required_df['text']=required_df['text'].apply(lambda x: clean_text(x))

In [ ]:
required_df.head()

,text,label,filename
0,rt uberdogfound please rt food needed for shel...,Positive,2011Tornado_Summary.csv
1,please rt food needed for shelter in nc in are...,Positive,2011Tornado_Summary.csv
2,its raining and tornado to be in a coma from e...,Positive,2011Tornado_Summary.csv
3,lmao damn rt lucecannonn if i don t get any fo...,Positive,2011Tornado_Summary.csv
4,saved by the tornado warning i m just ready to...,Negative,2011Tornado_Summary.csv


In [ ]:
train, test = train_test_split(required_df, test_size=0.7,stratify=required_df[['filename','label']],random_state=0)

In [ ]:
train

,text,label,filename
35066,1pm uk news brexit showdown hurricane dorian h...,Positive,2019Dorian_Summary.csv
22520,realdonaldtrump is bragging about crowd size a...,Negative,2017Hurricane_Summary.csv
30327,watch snowboarders ride down deserted streets ...,Positive,2016Blizzard_Summary.csv
27406,nc man shot and killed good samaritan trying t...,Positive,2016Blizzard_Summary.csv
39645,tv just went out hurricane dorian is here now ...,Negative,2019Dorian_Summary.csv
...,...,...,...
19018,off to the garage for a 1hr trainer ride befor...,Positive,2012Sandy_Summary.csv
38427,113 animals killed after shocking storm surge ...,Positive,2019Dorian_Summary.csv
27066,emergency mgmt gimme shelter keeping homeless ...,Positive,2016Blizzard_Summary.csv
12660,though the wildfires in california are now 100...,Positive,2018Wildfires_Summary.csv


In [ ]:
class MyClassificationDataset(Dataset):
    #'input_ids', 'attention_mask', 'label'
    def __init__(self, data):
        text, labels = data['text'].values,data['label'].values
        text = [str(i) for i in data['text'].values]
        self.examples = tokenizer(text,truncation=True,padding="max_length",
                                  return_tensors="pt")
        self.labels = tokenizer(text,truncation=True,padding="max_length",
                                  return_tensors="pt")
        

    def __len__(self):
        return len(self.examples["input_ids"])

    def __getitem__(self, index):
        return {'input_ids': self.examples['input_ids'][index],
                'attention_mask':self.examples['attention_mask'][index],
                'labels':self.labels[index]}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = [str(i) for i in  examples["text"].values]
    targets = [str(i) for i in  examples["label"].values]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

train_dataset=Dataset.from_dict(preprocess_function(train))
eval_dataset=Dataset.from_dict(preprocess_function(test))


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [ ]:
train_dataloader

In [ ]:

# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 3675/3675 [15:38<00:00,  3.92it/s]


epoch=0: train_ppl=tensor(1.2772, device='cuda:0') train_epoch_loss=tensor(0.2447, device='cuda:0') eval_ppl=tensor(1.1694, device='cuda:0') eval_epoch_loss=tensor(0.1565, device='cuda:0')


100%|██████████| 3675/3675 [15:39<00:00,  3.91it/s]


epoch=1: train_ppl=tensor(1.1741, device='cuda:0') train_epoch_loss=tensor(0.1605, device='cuda:0') eval_ppl=tensor(1.1496, device='cuda:0') eval_epoch_loss=tensor(0.1394, device='cuda:0')


100%|██████████| 3675/3675 [15:38<00:00,  3.91it/s]

epoch=2: train_ppl=tensor(1.1414, device='cuda:0') train_epoch_loss=tensor(0.1323, device='cuda:0') eval_ppl=tensor(1.1355, device='cuda:0') eval_epoch_loss=tensor(0.1271, device='cuda:0')


In [ ]:
eval_preds

In [1]:
# print accuracy
correct = 0
total = 0
for pred, true in zip(eval_preds, test["label"].values):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
#print(f"{dataset['validation']['text_label'][:10]=}")

NameError: ignored

In [ ]:
test['predictions']=eval_preds
test.to_csv("Lora_ndt_flan-t5-large_epochs_3_lr_1e-3.csv",index=False)

In [ ]:
np.unique(eval_preds)

array(['Neg', 'Negative', 'Positive', 'Positiveative'], dtype='<U13')